In [1]:
import torch
from torch import nn
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity
import cProfile

In [11]:
n = 2048
m = 4096
x = torch.rand(n, m)
linear = nn.Sequential(nn.Linear(m, n, bias=False).eval(), nn.Identity())
linear.weight.requires_grad = False
# linear.bias.requires_grad = False

with torch.inference_mode():
    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        linear(x)
print(prof.key_averages().table())

AttributeError: 'Sequential' object has no attribute 'weight'

jit = torch.jit.script(linear)
torch.jit.save(jit, 'linear.pt')

In [12]:
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfigMapping

with torch.inference_mode():
    torch.backends.quantized.engine = 'x86'
    qconfig = get_default_qconfig('x86')
    qconfig_mapping = QConfigMapping().set_global(qconfig)
    example_inputs = torch.randn_like(x)
    prepared_model = prepare_fx(linear, qconfig_mapping, example_inputs)
    for _ in range(16):
        prepared_model(torch.randn_like(x))

    quantized_model = convert_fx(prepared_model)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        quantized_model(x)

print(prof.key_averages().table())
print(quantized_model.code)

/home/alexey/.local/lib/python3.10/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        aten::quantize_per_tensor         2.43%       3.667ms         2.44%       3.676ms       3.676ms             1  
                       aten::item         0.00%       7.000us         0.01%       9.000us       4.500us             2  
        aten::_local_scalar_dense         0.00%       2.000us         0.00%       2.000us       1.000us             2  
                quantized::linear        94.52%     142.649ms        94.84%     143.131ms     143.131ms             1  
    aten::_empty_affine_quantized         0.01%      11.000us         0.01%      11.000us      11.000us             1  
                    aten::q_scale       

STAGE:2023-12-10 17:08:27 55896:55896 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-12-10 17:08:27 55896:55896 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-12-10 17:08:27 55896:55896 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [7]:
# torch.jit.save(jit, f'x86_linear_{n}x{m}.pt')

In [8]:
jit = torch.jit.script(quantized_model)
print(jit.code)
torch.jit.save(jit, f'x86_linear_{n}x{m}.pt')

def forward(self,
    input: Tensor) -> Tensor:
  _input_scale_0 = self._input_scale_0
  _input_zero_point_0 = self._input_zero_point_0
  quantize_per_tensor = torch.quantize_per_tensor(input, _input_scale_0, _input_zero_point_0, 13)
  _packed_weight_0 = self._packed_weight_0
  _scale_1 = self._scale_1
  _zero_point_1 = self._zero_point_1
  linear = ops.quantized.linear(quantize_per_tensor, _packed_weight_0, annotate(float, _scale_1), annotate(int, _zero_point_1))
  return torch.dequantize(linear)



/home/alexey/.local/lib/python3.10/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


In [9]:
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfigMapping

with torch.inference_mode():
    torch.backends.quantized.engine = 'fbgemm'
    qconfig = get_default_qconfig('fbgemm')
    qconfig_mapping = QConfigMapping().set_global(qconfig)
    example_inputs = torch.randn_like(x)
    prepared_model = prepare_fx(linear, qconfig_mapping, example_inputs)
    for _ in range(16):
        prepared_model(torch.randn_like(x))

    quantized_model = convert_fx(prepared_model)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        quantized_model(x)

print(prof.key_averages().table())
print(quantized_model.code)

/home/alexey/.local/lib/python3.10/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        aten::quantize_per_tensor         2.47%       2.741ms         2.48%       2.751ms       2.751ms             1  
                       aten::item         0.02%      18.000us         0.02%      24.000us       6.000us             4  
        aten::_local_scalar_dense         0.01%       7.000us         0.01%       7.000us       1.750us             4  
                quantized::linear        95.29%     105.915ms        95.32%     105.951ms     105.951ms             1  
    aten::_empty_affine_quantized         0.01%      16.000us         0.01%      16.000us      16.000us             1  
                    aten::q_scale       

/home/alexey/.local/lib/python3.10/site-packages/torch/fx/graph.py:1377: UserWarning: Node _packed_weight_0 target _packed_weight_0 _packed_weight_0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
STAGE:2023-12-10 16:55:33 55896:55896 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-12-10 16:55:34 55896:55896 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-12-10 16:55:34 55896:55896 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [10]:
jit = torch.jit.script(quantized_model)
print(jit.code)
torch.jit.save(jit, f'fbgemm_linear_{n}x{m}.pt')

def forward(self,
    input: Tensor) -> Tensor:
  _input_scale_0 = self._input_scale_0
  _input_zero_point_0 = self._input_zero_point_0
  quantize_per_tensor = torch.quantize_per_tensor(input, _input_scale_0, _input_zero_point_0, 13)
  _packed_weight_0 = self._packed_weight_0
  _scale_1 = self._scale_1
  _zero_point_1 = self._zero_point_1
  linear = ops.quantized.linear(quantize_per_tensor, _packed_weight_0, annotate(float, _scale_1), annotate(int, _zero_point_1))
  return torch.dequantize(linear)



/home/alexey/.local/lib/python3.10/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


In [6]:
print(jit.graph)

graph(%self : __torch__.torch.fx.graph_module.GraphModule,
      %input.1 : Tensor):
  %8 : int = prim::Constant[value=13]() # <eval_with_key>.15:8:98
  %_input_scale_0.1 : Tensor = prim::GetAttr[name="_input_scale_0"](%self)
  %_input_zero_point_0.1 : Tensor = prim::GetAttr[name="_input_zero_point_0"](%self)
  %quantize_per_tensor.1 : Tensor = aten::quantize_per_tensor(%input.1, %_input_scale_0.1, %_input_zero_point_0.1, %8) # <eval_with_key>.15:8:26
  %_packed_weight_0.1 : __torch__.torch.classes.quantized.LinearPackedParamsBase = prim::GetAttr[name="_packed_weight_0"](%self)
  %_scale_1.1 : Tensor = prim::GetAttr[name="_scale_1"](%self)
  %_zero_point_1.1 : Tensor = prim::GetAttr[name="_zero_point_1"](%self)
  %21 : float = aten::FloatImplicit(%_scale_1.1) # <eval_with_key>.15:12:13
  %22 : int = aten::IntImplicit(%_zero_point_1.1) # <eval_with_key>.15:12:13
  %linear.1 : Tensor = quantized::linear(%quantize_per_tensor.1, %_packed_weight_0.1, %21, %22) # <eval_with_key>.15:12:13
  %

In [7]:
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfigMapping

conv2d = nn.Conv2d(16, 8, 3)
image = torch.randn(1, 16, 64, 64)

with torch.inference_mode():

    qconfig = get_default_qconfig('fbgemm')
    qconfig_mapping = QConfigMapping().set_global(qconfig)
    example_inputs = torch.randn_like(image)
    prepared_model = prepare_fx(conv2d, qconfig_mapping, example_inputs)
    for _ in range(16):
        prepared_model(torch.randn_like(image))

    quantized_model = convert_fx(prepared_model)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        quantized_model(image)

print(prof.key_averages().table())
print(quantized_model.code)

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        aten::quantize_per_tensor         0.23%      60.000us         0.26%      68.000us      68.000us             1  
                       aten::item         0.03%       9.000us         0.04%      11.000us       2.750us             4  
        aten::_local_scalar_dense         0.01%       2.000us         0.01%       2.000us       0.500us             4  
                quantized::conv2d        35.61%       9.285ms        36.00%       9.387ms       9.387ms             1  
                 aten::contiguous         0.01%       2.000us         0.36%      95.000us      95.000us             1  
                      aten::clone       

STAGE:2023-11-20 19:58:43 112085:112085 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-11-20 19:58:43 112085:112085 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-11-20 19:58:43 112085:112085 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
